# Beyond Classical gradient descent : What to expect from different Optimizers


Most the time when I train Neural Networks on tensorflow I set the optimizer by default to Adam optimizer. I have already look into formula but I have never implemented any of them. Here is some simple code. 

##  Model definition

We define a three layer Neural Network and some operations to do our back propagation algorithm.

In [24]:
from sklearn.datasets import make_moons
from sklearn.cross_validation import train_test_split
import numpy as np
from sklearn.utils import shuffle
from IPython.display import display, Math, Latex

Dataset definition : we use the make_moon dataset

In [2]:
X, y = make_moons(n_samples=5000, random_state=42, noise=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Model definition : 3 layers NN

In [3]:
n_feature = 2
n_class = 2
n_iter = 10
def make_network(n_hidden=100):
    # Initialize weights with Standard Normal random variables
    model = dict(
        W1=np.random.randn(n_feature, n_hidden),
        W2=np.random.randn(n_hidden, n_class)
    )
    return model

Minibatch function to get a random sample of a dataset.

In [4]:
def get_minibatch(X, y, minibatch_size):
    minibatches = []

    X, y = shuffle(X, y)

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]

        minibatches.append((X_mini, y_mini))

    return minibatches

The following function will be used for training the Neural Network :

- softmax function
- backward : start by propagating the error to get the gradient of weight between hidden layer and output layer, then we compute the gradient of hidden layer using the error gradient. Using hidden layer gradient, we then compute the gradient of weight between input and hidden layer.
- forward : we do series of dot product from input to hidden layer to output layer

In [5]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum()


def forward(x, model):
    # Input to hidden
    h = x @ model['W1']
    # ReLU non-linearity
    h[h < 0] = 0

    # Hidden to output
    prob = softmax(h @ model['W2'])

    return h, prob

def backward(model, xs, hs, errs):
    """xs, hs, errs contain all informations (input, hidden state, error) of all data in the minibatch"""
    # errs is the gradients of output layer for the minibatch
    dW2 = hs.T @ errs

    # Get gradient of hidden layer
    dh = errs @ model['W2'].T
    dh[hs <= 0] = 0

    dW1 = xs.T @ dh

    return dict(W1=dW1, W2=dW2)

Compute the basic gradient

In [6]:
def get_minibatch_grad(model, X_train, y_train):
    xs, hs, errs = [], [], []

    for x, cls_idx in zip(X_train, y_train):
        h, y_pred = forward(x, model)

        # Create probability distribution of true label
        y_true = np.zeros(n_class)
        y_true[int(cls_idx)] = 1.

        # Compute the gradient of output layer
        err = y_true - y_pred

        # Accumulate the informations of minibatch
        # x: input
        # h: hidden state
        # err: gradient of output layer
        xs.append(x)
        hs.append(h)
        errs.append(err)

    # Backprop using the informations we get from the current minibatch
    return backward(model, np.array(xs), np.array(hs), np.array(errs))

## Basic mini-batch gradient descent

This is the simplest algorithm. Instead of computing the gradient with all the training sample we use only a small amout of sample. 

In [7]:
def sgd(model, X_train, y_train, minibatch_size):
    minibatches = get_minibatch(X_train, y_train, minibatch_size)

    for iter in range(1, n_iter + 1):
        idx = np.random.randint(0, len(minibatches))
        X_mini, y_mini = minibatches[idx]

        grad = get_minibatch_grad(model, X_mini, y_mini)

        for layer in grad:
            model[layer] += alpha * grad[layer]

    return model

## SGD + Momentum

The idea behind Momentum is to gain into speed when we go downhill not to be stuck into a wrong minima (if we gain speed we will be able to pass an other uphill)

\begin{align}
\nabla \times \vec{\mathbf{B}} -\, \frac1c\, \frac{\partial\vec{\mathbf{E}}}{\partial t} & = \frac{4\pi}{c}\vec{\mathbf{j}} \\
\nabla \cdot \vec{\mathbf{E}} & = 4 \pi \rho \\
\nabla \times \vec{\mathbf{E}}\, +\, \frac1c\, \frac{\partial\vec{\mathbf{B}}}{\partial t} & = \vec{\mathbf{0}} \\
\nabla \cdot \vec{\mathbf{B}} & = 0
\end{align}


In [8]:
def momentum(model, X_train, y_train, minibatch_size):
    velocity = {k: np.zeros_like(v) for k, v in model.items()}
    gamma = .9

    minibatches = get_minibatch(X_train, y_train, minibatch_size)

    for iter in range(1, n_iter + 1):
        idx = np.random.randint(0, len(minibatches))
        X_mini, y_mini = minibatches[idx]

        grad = get_minibatch_grad(model, X_mini, y_mini)

        for layer in grad:
            velocity[layer] = gamma * velocity[layer] + alpha * grad[layer]
            model[layer] += velocity[layer]

    return model

## Nesterov Momentum

 Nesterov Momentum add one little different bit to the momentum calculation. Instead of calculating gradient of the current position, it calculates the gradient at the approximated new position.

In [9]:
def nesterov(model, X_train, y_train, minibatch_size):
    velocity = {k: np.zeros_like(v) for k, v in model.items()}
    gamma = .9

    minibatches = get_minibatch(X_train, y_train, minibatch_size)

    for iter in range(1, n_iter + 1):
        idx = np.random.randint(0, len(minibatches))
        X_mini, y_mini = minibatches[idx]

        model_ahead = {k: v + gamma * velocity[k] for k, v in model.items()}
        grad = get_minibatch_grad(model, X_mini, y_mini)

        for layer in grad:
            velocity[layer] = gamma * velocity[layer] + alpha * grad[layer]
            model[layer] += velocity[layer]

    return model

## Adagrad

The Adagrad uses a different approach : the learning rate is adaptive per-parameter. We accumulate the sum of squared of all of our parameters’ gradient, and use that to normalize the learning rate alpha

In [10]:
def adagrad(model, X_train, y_train, minibatch_size):
    cache = {k: np.zeros_like(v) for k, v in model.items()}

    minibatches = get_minibatch(X_train, y_train, minibatch_size)

    for iter in range(1, n_iter + 1):
        idx = np.random.randint(0, len(minibatches))
        X_mini, y_mini = minibatches[idx]

        grad = get_minibatch_grad(model, X_mini, y_mini)

        for k in grad:
            cache[k] += grad[k]**2
            model[k] += alpha * grad[k] / (np.sqrt(cache[k]) + eps)

    return model


## RMSprop

The only difference compared to Adagrad is how we calculate the cache,  this is to fight the monotically increasing gradient. Here, we’re take gamma portion of past accumulated sum of squared gradient, and take 1 - gamma portion of the current squared gradient.

In [11]:
def rmsprop(model, X_train, y_train, minibatch_size):
    cache = {k: np.zeros_like(v) for k, v in model.items()}
    gamma = .9

    minibatches = get_minibatch(X_train, y_train, minibatch_size)

    for iter in range(1, n_iter + 1):
        idx = np.random.randint(0, len(minibatches))
        X_mini, y_mini = minibatches[idx]

        grad = get_minibatch_grad(model, X_mini, y_mini)

        for k in grad:
            cache[k] = gamma * cache[k] + (1 - gamma) * (grad[k]**2)
            model[k] += alpha * grad[k] / (np.sqrt(cache[k]) + eps)

    return model

## Adam

Adam is RMSprop with momentum. So, Adam tries to combine the best of both world of momentum and adaptive learning rate.

In [12]:
def adam(model, X_train, y_train, minibatch_size):
    M = {k: np.zeros_like(v) for k, v in model.items()}
    R = {k: np.zeros_like(v) for k, v in model.items()}
    beta1 = .9
    beta2 = .999

    minibatches = get_minibatch(X_train, y_train, minibatch_size)

    for iter in range(1, n_iter + 1):
        t = iter
        idx = np.random.randint(0, len(minibatches))
        X_mini, y_mini = minibatches[idx]

        grad = get_minibatch_grad(model, X_mini, y_mini)

        for k in grad:
            M[k] = beta1 * M[k] + (1. - beta1) * grad[k]
            R[k] = beta2 * R[k] + (1. - beta2) * grad[k]**2

            m_k_hat = M[k] / (1. - beta1**(t))
            r_k_hat = R[k] / (1. - beta2**(t))

            model[k] += alpha * m_k_hat / (np.sqrt(r_k_hat) + eps)

    return model

## Testing and comparing gradient algorithms

In [13]:
def gradient_method(method,model, X_train, y_train, minibatch_size) :
    if method == "adam" :
        return adam(model, X_train, y_train, minibatch_size)
    if method == "rmsprop" :
        return rmsprop(model, X_train, y_train, minibatch_size)
    if method == "adagrad" :
        return adagrad(model, X_train, y_train, minibatch_size)
    if method == "nesterov" :
        return nesterov(model, X_train, y_train, minibatch_size)
    if method == "momentum" :
        return momentum(model, X_train, y_train, minibatch_size)
    if method == "sgd" :
        return sgd(model, X_train, y_train, minibatch_size)

In [21]:
def compute_accuracy(method, alpha) :

    # Create placeholder to accumulate prediction accuracy
    accs = np.zeros(n_experiment)

    for k in range(n_experiment):
        # Reset model
        model = make_network()

        # Train the model
        model = gradient_method(method,model, X_train, y_train, minibatch_size)

        y_pred = np.zeros_like(y_test)

        for i, x in enumerate(X_test):
            # Predict the distribution of label
            _, prob = forward(x, model)
            # Get label by picking the most probable one
            y = np.argmax(prob)
            y_pred[i] = y

        # Compare the predictions with the true labels and take the percentage
        accs[k] = (y_pred == y_test).sum() / y_test.size
    print('Mean {} accuracy: {}, std: {}'.format(method,accs.mean(), accs.std()))

In [22]:
n_iter = 100
eps = 1e-8  # Smoothing to avoid division by zero
minibatch_size = 50
n_experiment = 10
for alpha in [0.1,0.01,0.001] :
    print('------ Alpha value ------',alpha)
    for method in ["adam","rmsprop","adagrad","nesterov","momentum","sgd"] :
        compute_accuracy(method, alpha)
        

------ Alpha value ------ 0.1
Mean adam accuracy: 0.8749600000000001, std: 0.007029537680388374
Mean rmsprop accuracy: 0.8668000000000001, std: 0.01761181421659905
Mean adagrad accuracy: 0.8772, std: 0.003283900120283805


/Users/pierrecolombo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/pierrecolombo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/Users/pierrecolombo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in less
  if __name__ == '__main__':
/Users/pierrecolombo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in less_equal


Mean nesterov accuracy: 0.4768, std: 0.0
Mean momentum accuracy: 0.4768, std: 0.0
Mean sgd accuracy: 0.6372, std: 0.19645376046286311
------ Alpha value ------ 0.01
Mean adam accuracy: 0.8787200000000001, std: 0.001394847661933016
Mean rmsprop accuracy: 0.86488, std: 0.024303283728747457
Mean adagrad accuracy: 0.8618399999999999, std: 0.04239856601348682
Mean nesterov accuracy: 0.87104, std: 0.02121976437192457
Mean momentum accuracy: 0.8759999999999998, std: 0.005645529204600758
Mean sgd accuracy: 0.8317599999999998, std: 0.05985481100128878
------ Alpha value ------ 0.001
Mean adam accuracy: 0.8254400000000001, std: 0.07495944503529892
Mean rmsprop accuracy: 0.82944, std: 0.08210806537728191
Mean adagrad accuracy: 0.50504, std: 0.23532705411830573
Mean nesterov accuracy: 0.8734400000000001, std: 0.010611993215225874
Mean momentum accuracy: 0.8766399999999999, std: 0.0031556932677305515
Mean sgd accuracy: 0.87768, std: 0.0020380382724571105
